In [9]:
use std::fs::File;
use std::io::{BufReader, BufRead, Lines};
use std::collections::HashMap;
use std::ops::{Deref, DerefMut, Sub, Add};

In [10]:
let file = File::open("c_many_ingredients.in");
let reader = BufReader::new(file.expect("Cannot open file"));
let mut lines = reader.lines();

In [11]:
let first_line = lines.next().unwrap().unwrap();
let infos = first_line.trim().split(' ').map(|val| val.parse::<u32>().unwrap()).collect::<Vec<_>>();

let pizzas = infos[0];
let nb_2 = infos[1];
let nb_3 = infos[2];
let nb_4 = infos[3];

In [12]:
struct HashMapIterator(HashMap<&'static str, u16>, Lines<BufReader<File>>, u16);

impl HashMapIterator {
    fn new(lines: Lines<BufReader<File>>) -> HashMapIterator {
        HashMapIterator(HashMap::new(), lines, 0)
    }
    
    fn get_hash_map(self) -> HashMap<&'static str, u16> {
        self.0
    }
    
    fn consume(mut self) -> (HashMap<&'static str, u16>, Vec<Pizza>) {
        let res = (&mut self).collect::<Vec<Pizza>>();
        (self.0, res)
    }
}

impl Iterator for HashMapIterator {
    type Item = Pizza;
    
    fn next(&mut self) -> Option<Pizza> {
        let line = match self.1.next() {
            Some(line) => line.expect("Cannot read file"),
            None => return None,
        };
        let mut line_split = line.split(' ');
        let mut ingredients = Vec::with_capacity(line_split.next().unwrap().parse().unwrap());
        for ingredient in line_split {
            ingredients.push(
                if self.0.contains_key(ingredient){
                    *self.0.get(ingredient).unwrap()
                } else {
                    self.0.insert(Box::leak(ingredient.to_string().into_boxed_str()), self.2);
                    self.2 += 1;
                    self.2 - 1
                }
            )
        }
        Some(Pizza::new(ingredients))
    }
}

#[derive(Debug)]
struct Pizza(Vec<u16>);

impl Pizza {
    fn new(mut ingredients: Vec<u16>) -> Pizza {
        ingredients.sort();
        Pizza(ingredients)
    }
    
    fn fusion_diff(lhs: &[u16], rhs: &[u16], acc: usize) -> usize {
        if lhs.len() == 0 {
            rhs.len() + acc
        } else if rhs.len() == 0 {
            lhs.len() + acc
        } else {
            if lhs[0] == rhs[0] {
                Self::fusion_diff(&lhs[1..], &rhs[1..], acc)
            } else if lhs[0] < rhs[0] {
                Self::fusion_diff(&lhs[1..], rhs, acc + 1)
            } else {
                Self::fusion_diff(lhs, &rhs[1..], acc + 1)
            }
        }
    }
    
    fn fusion_add(lhs: &[u16], rhs: &[u16], mut acc: Pizza) -> Pizza {
        if lhs.len() == 0 {
            rhs.iter().for_each(|&ingredient| acc.push(ingredient));
            acc
        } else if rhs.len() == 0 {
            lhs.iter().for_each(|&ingredient| acc.push(ingredient));
            acc
        } else {
            if lhs[0] == rhs[0] {
                acc.push(lhs[0]);
                Self::fusion_add(&lhs[1..], &rhs[1..], acc)
            } else if lhs[0] < rhs[0] {
                acc.push(lhs[0]);
                Self::fusion_add(&lhs[1..], rhs, acc)
            } else {
                acc.push(rhs[0]);
                Self::fusion_add(lhs, &rhs[1..], acc)
            }
        }
    }
}

impl Deref for Pizza {
    type Target = Vec<u16>;
    
    fn deref(&self) -> &Vec<u16> {
        &self.0
    }
}

impl DerefMut for Pizza {
    fn deref_mut(&mut self) -> &mut Vec<u16> {
        &mut self.0
    }
}



impl Sub<&Pizza> for &Pizza {
    type Output = usize;
    
    fn sub(self, rhs: &Pizza) -> usize {
        Pizza::fusion_diff(&self[..], &rhs[..], 0)
    }
}

impl Add<&Pizza> for &Pizza {
    type Output = Pizza;
    
    fn add(self, rhs: &Pizza) -> Pizza {
        Pizza::fusion_add(&self[..], &rhs[..], Pizza::new(Vec::with_capacity(self.len())))
    }
}



In [13]:
let res = HashMapIterator::new(lines).consume().1;
//res

In [109]:
let diff = res[..res.len() - 1].iter().enumerate().map( |(idx, pizza)| (0..=0).cycle().take(idx + 1).chain(res[idx + 1..].iter().map(move |pizza2| pizza - pizza2))).flatten().collect::<Vec<_>>();

In [73]:
diff_vec.iter().enumerate().fold((0, 0), |(idx_max, max), (idx, &elem)| {
    if elem > max {
        (idx, elem)
    } else {
        (idx_max, max)
    }
}).0

1

In [84]:
diff

[0, 6, 4, 6, 5, 0, 0, 4, 0, 3, 0, 0, 0, 4, 3, 0, 0, 0, 0, 3]

In [18]:
res[..res.len() - 1].iter().enumerate().fold((0, 0, 0), |(i, j, max), (idx1, pizza1)| {
    res[idx1 + 1..].iter().enumerate().fold((i, j, max), move |(i, j, max), (idx2, pizza2)| {
        if pizza1.len() + pizza2.len() > max {
            let new_max = pizza1 - pizza2; 
            if new_max > max {
                return (idx1, idx2 + idx1 + 1, new_max)
            }
        }
        (i, j,max)
    })
})

(5121, 5357, 1763)

In [19]:
use std::collections::BinaryHeap;
use std::cmp::Reverse;

10000